## DATA PREPROCESSING

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix

In [17]:
df = pd.read_excel("category.xlsx",sheet_name= [1,4])
loc_to_cat = df[1]
loc_to_cat['category1'] = loc_to_cat['category1'].str.lower()
allUserData = df[4]
#userData = [df[3], df[4], df[5], df[6], df[7]]

#for i in userData[1:]:
#    allUserData = allUserData.append(i)
allUserData.head()

,user_id,weekday,date,start_time,end_date,end_time,duration,category,category_id,location_name,location_category,my_place,location_id,place_id
0,2,2,2020-06-29,16:00:00,2020-06-29,19:00:00,03:00:00,meeting,13,路易莎,cafe,NaN,4.0,NaN
1,2,2,2020-06-29,19:00:00,2020-06-29,20:00:00,01:00:00,food,1,全哥鵝肉小炒,restaurant,NaN,5.0,NaN
2,2,2,2020-06-29,20:00:00,2020-06-29,20:30:00,00:30:00,leisure,10,NaN,NaN,home,NaN,1.0
3,2,2,2020-06-29,20:30:00,2020-06-29,21:30:00,01:00:00,shopping,12,Miramar,Shopping mall,NaN,6.0,NaN
4,2,2,2020-06-29,21:30:00,2020-06-29,23:30:00,02:00:00,study,5,NaN,NaN,home,NaN,1.0


In [18]:
allUserData['location']= allUserData['location_id'].combine_first(allUserData['place_id'])
#drop commute
indexNames = allUserData[(allUserData['category_id'] == 17) ].index
allUserData.drop(indexNames,inplace=True)

allUserData['location'].fillna("99", inplace = True)
allUserData = allUserData.drop(['date','end_date','end_time','location_name','location_category','my_place','location_id','place_id'], 1)
allUserData = allUserData.reset_index(drop = True)

In [19]:
def get_hours(ts):
    a = ts.hour
    return a

def get_mins(ts):
    a = ts.minute
    return a

def get_duration(ts):
    a = ts.hour*60 + ts.minute
    return a 

#allUserData['start_mins'] = allUserData['start_time'].apply(get_mins)
allUserData['start_time'] = allUserData['start_time'].apply(get_duration)
allUserData['duration'] = allUserData['duration'].apply(get_duration)
allUserData = allUserData[['user_id','weekday','start_time','duration','location','category','category_id']]
allUserData.tail()

,user_id,weekday,start_time,duration,location,category,category_id
444,2,5,50,495,1,sleep,3
445,2,5,545,15,1,leisure,10
446,2,5,570,150,2,work,6
447,2,5,780,360,3,meeting,13
448,2,5,1200,240,1,leisure,10


## Random Forest Model

In [28]:
train_df = allUserData.loc[0:299]
test_df = allUserData.loc[300:]
print("Number of traing data is :",len(train_df))
print("Number of testing data is :",len(test_df))

Number of traing data is : 300
Number of testing data is : 149


In [29]:
# without diff
X_train = train_df.iloc[:, np.r_[0:4]].values
y_train = train_df.iloc[:, 6].values

X_test = test_df.iloc[:, np.r_[0:4]].values
y_test = test_df.iloc[:, 6].values

In [30]:
clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=50, verbose=0,
            warm_start=False)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=50, verbose=0, warm_start=False)

In [31]:
clf.score(X_test, y_test)

0.6308724832214765

In [32]:
imp = clf.feature_importances_
imp

array([0.        , 0.11707604, 0.48461265, 0.3983113 ])

In [26]:
predictions = clf.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           1       0.36      0.71      0.48        21
           3       1.00      0.95      0.97        19
           4       0.00      0.00      0.00         0
           6       0.50      0.82      0.62        11
           9       0.80      0.80      0.80         5
          10       0.80      0.71      0.75        58
          11       0.50      0.12      0.20         8
          12       0.44      0.25      0.32        16
          13       1.00      0.33      0.50         6
          16       0.00      0.00      0.00         2
          18       0.00      0.00      0.00         3

   micro avg       0.63      0.63      0.63       149
   macro avg       0.49      0.43      0.42       149
weighted avg       0.67      0.63      0.62       149



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [27]:
print(confusion_matrix(y_test,predictions))

[[15  0  0  0  0  4  1  1  0  0  0]
 [ 0 18  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  9  0  1  0  0  0  1  0]
 [ 0  0  0  0  4  1  0  0  0  0  0]
 [ 8  0  1  5  0 41  0  3  0  0  0]
 [ 5  0  0  0  0  2  1  0  0  0  0]
 [10  0  0  0  1  1  0  4  0  0  0]
 [ 0  0  0  4  0  0  0  0  2  0  0]
 [ 2  0  0  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  1  0  0  0]]


In [33]:
predictions

array([12,  1,  1, 10, 10, 10,  3, 10, 10,  1, 10, 11,  1,  1,  4, 10,  3,
       10,  6, 10, 10, 10, 10,  1,  6,  1, 12, 10, 10,  3,  6,  6, 10,  9,
       10,  3,  6,  1,  1, 13, 10, 10,  9, 10,  3,  6,  6,  1, 10, 10,  3,
       10,  1,  1, 12,  1,  1, 11,  1, 10,  3, 10,  3, 10, 12, 12,  1,  6,
        1,  1, 12,  1, 10,  3, 10,  6,  1, 10, 12,  9, 10,  3,  6,  6,  1,
       10,  9, 10, 10,  3, 10,  6,  1, 13,  1,  1,  1, 10, 10,  3, 10,  1,
       16,  1, 10,  3, 10,  1,  1, 10,  1,  1,  1, 10,  3, 10,  1, 10, 12,
       10,  3, 10,  6,  1, 12, 10,  3, 10,  1,  6,  1, 10,  3,  6,  6,  1,
        1,  1,  1,  1,  1, 10,  9, 10,  3, 10,  6,  6, 10])

## User Data

In [432]:
location = pd.read_csv("0909.csv")
location.tail()

,location_id,longitude,latitude,start_date,start_time,weekday,duration,speed,name1,name2,name3,name4,name5,category1,category2,category3,category4,category5
406,407,121.559666,25.084203,2020-09-09,21:29,4,0.0,-1.0,World Gym世界健身俱樂部 台北大直店Sport,Grand Victoria Hotel Taipei,N°168 Prime Steakhouse 牛排館,Hotel Indigo Taipei North,福莎足球場,gym,lodging,restaurant,lodging,point_of_interest
407,408,121.560135,25.085544,2020-09-09,22:50,4,0.0,-1.0,CPC,Wen Hu Elementary School,全聯福利中心Pxmart 內湖文湖,丹特 凹痕回復專門店,來來豆漿,gas_station,primary_school,supermarket,car_repair,restaurant
408,409,25.087294,25.087294,2020-09-09,22:50,4,0.0,-1.0,家,4,NaN,NaN,NaN,home,entering myPlace,NaN,NaN,NaN
409,410,121.560943,25.086228,2020-09-09,22:50,4,0.0,-1.0,全聯福利中心Pxmart 內湖文湖,Wen Hu Elementary School,建安輪胎有限公司,米其林馳加汽車服務中心(建凱店),Hilife 萊爾富,supermarket,primary_school,car_repair,car_repair,convenience_store
410,411,121.562773,25.086631,2020-09-09,22:58,4,0.0,-1.0,Family Mart,小鳥不吃肉,全聯福利中心Pxmart 內湖文湖,佳佳義大利麵,嘉儀電器,convenience_store,restaurant,supermarket,restaurant,store


In [433]:
stopLoc = location.drop(['category3','category4','category5'], 1)

# def string_to_mins(ts):
#     a = int(ts.split(':')[0])
#     b = int(ts.split(':')[1])
#     c = a*60 + b
#     return c

#stopLoc['start_time'] = pd.to_datetime(stopLoc['start_time'], format='%H:%M', errors='ignore').apply(string_to_mins)
stopLoc['longitude'] = round(stopLoc['longitude'],3)
stopLoc['latitude'] = round(stopLoc['latitude'],3)

In [434]:
enterList = []
dropList = []

for a in range(len(stopLoc)):
    
    #如果這筆是「enter」，記住編號
    if (stopLoc.iloc[a,14] == 'entering myPlace'):
        enterList.append(a)
                
    #如果這筆是「exit」，清空enterList
    elif (stopLoc.iloc[a,14] == 'exiting myPlace'):
        enterList = []
            
    elif (enterList == []):
        pass
            
    #如果這筆不是進去也不是出來，且有enterList，為要刪除的東西
    elif (enterList != []):
        dropList.append(a) 

stopLoc = stopLoc.drop(dropList)
stopLoc = stopLoc.reset_index(drop = True)   

In [435]:
#delete trivial data

dropList = []
for row in range(1,len(stopLoc)-1): 
    if stopLoc.iloc[row,14] != 'entering myPlace' and stopLoc.iloc[row,14] != 'exiting myPlace':
        
        #same longitude and lantitude
        if stopLoc.iloc[row,1] == stopLoc.iloc[row-1,1] and stopLoc.iloc[row,2] == stopLoc.iloc[row-1,2]:
            dropList.append(row)

        #compared with the last record, over 2 place name duplicated will be deleted
        res = set(stopLoc.iloc[row,8:13]) & set(stopLoc.iloc[row-1,8:13])
        if len(res) > 1: 
            dropList.append(row)

stopLoc = stopLoc.drop(dropList)
stopLoc = stopLoc.reset_index(drop = True)

In [436]:
#kill duration < 10 mins
import datetime as dt
from datetime import datetime

dropList = []
for row in range(0,len(stopLoc)-1): 

    #calculate duration
    if stopLoc.iloc[row+1,3] == stopLoc.iloc[row,3]:
        stopLoc.iloc[row,6] = datetime.strptime(stopLoc.iloc[row+1,4], '%H:%M') - datetime.strptime(stopLoc.iloc[row,4], '%H:%M')       
    else:
        a = dt.datetime(2000,1,2,int(stopLoc.iloc[row+1,4].split(':')[0]),int(stopLoc.iloc[row+1,4].split(':')[1]))
        b = dt.datetime(2000,1,1,int(stopLoc.iloc[row,4].split(':')[0]),int(stopLoc.iloc[row,4].split(':')[1]))
        stopLoc.iloc[row,6] = a - b
            
    #kill duration <10
    if pd.Timedelta(stopLoc.iloc[row,6]) < pd.Timedelta('00:10:00'):
        if stopLoc.iloc[row,14] != 'entering myPlace' and stopLoc.iloc[row,14] != 'exiting myPlace':
            dropList.append(row)    
            
removeList = []
for i in range(1,len(dropList)-1):
    if dropList[i] - dropList[i-1] < 3:ㄋ
        removeList.append(dropList[i])

stopLoc = stopLoc.drop(removeList)
stopLoc = stopLoc.reset_index(drop = True)

In [437]:
#delete duplicate
duplicate = []
for row in range(0,len(stopLoc)-1): 
    
    if (stopLoc.iloc[row,14] == 'exiting myPlace') and (stopLoc.iloc[row+2,14] == 'entering myPlace'):
        if (stopLoc.iloc[row,8] == stopLoc.iloc[row+2,8]):
            duplicate.extend([row,row+1,row+2])
    
    #kill duration <10
    if pd.Timedelta(stopLoc.iloc[row,6]) < pd.Timedelta('00:10:00'):
        if stopLoc.iloc[row,14] != 'entering myPlace' and stopLoc.iloc[row,14] != 'exiting myPlace':
            duplicate.append(row)  

stopLoc = stopLoc.drop(duplicate)
stopLoc = stopLoc.reset_index(drop = True)
            
for row in range(0,len(stopLoc)-1): 
    #calculate duration
    if stopLoc.iloc[row+1,3] == stopLoc.iloc[row,3]:
        stopLoc.iloc[row,6] = datetime.strptime(stopLoc.iloc[row+1,4], '%H:%M') - datetime.strptime(stopLoc.iloc[row,4], '%H:%M')       
    else:
        a = dt.datetime(2000,1,2,int(stopLoc.iloc[row+1,4].split(':')[0]),int(stopLoc.iloc[row+1,4].split(':')[1]))
        b = dt.datetime(2000,1,1,int(stopLoc.iloc[row,4].split(':')[0]),int(stopLoc.iloc[row,4].split(':')[1]))
        stopLoc.iloc[row,6] = a - b
          
stopLoc = stopLoc.drop(['start_date','name2','name3','name4','name5','speed'], 1)
stopLoc

,location_id,longitude,latitude,start_time,weekday,duration,name1,category1,category2
0,1,121.576,24.986,11:00,4,"1 day, 1:00:00",myhome,c1,c2
1,2,121.443,24.433,12:00,5,"1 day, 1:00:00",apple,c1,c2
2,3,121.123,24.765,13:00\n,6,"1 day, 5:00:00",banana,c1,c2
3,4,121.522,25.063,18:00,3,1:14:00,LOUISA COFFEE 路易莎咖啡 (民權西門市),cafe,restaurant
4,57,25.087,25.087,19:14,3,17:04:00,家,home,entering myPlace
5,164,121.563,25.087,12:18,4,0:34:00,家,home,exiting myPlace
6,244,25.057,25.057,12:52,4,4:02:00,公司,work place,entering myPlace
7,267,121.616,25.057,16:54,4,0:28:00,公司,work place,exiting myPlace
8,374,25.087,25.087,17:22,4,1:30:00,家,home,entering myPlace
9,388,121.563,25.087,18:52,4,0:08:00,家,home,exiting myPlace


In [438]:
twtymin = pd.Timedelta('00:20:00')

for row in range(0,len(stopLoc)-1): 

    if stopLoc.iloc[row,8] != 'entering myPlace':
        #mark as commute if distance bigger than 500 meters. 
#         if (abs(stopLoc.iloc[row+1,1] - stopLoc.iloc[row,1]) + abs(stopLoc.iloc[row+1,2] - stopLoc.iloc[row,2])) > 0.02:
#             print(row)
#             stopLoc.iloc[row,7] = 'commute'
            
#         elif (abs(stopLoc.iloc[row-1,1] - stopLoc.iloc[row,1]) + abs(stopLoc.iloc[row-1,2] - stopLoc.iloc[row,2])) > 0.02:
#             stopLoc.iloc[row,7] = 'commute'
        if stopLoc.iloc[row,8] == 'exiting myPlace':
            stopLoc.iloc[row,7] = 'commute' 
            
        if pd.Timedelta(stopLoc.iloc[row-1,5]) > twtymin :
            if abs(stopLoc.iloc[row-1,1] - stopLoc.iloc[row,1]) > 0.02 or abs(stopLoc.iloc[row-1,2] - stopLoc.iloc[row,2]) > 0.02:
                stopLoc.iloc[row,7] = 'commute'        
        
stopLoc

,location_id,longitude,latitude,start_time,weekday,duration,name1,category1,category2
0,1,121.576,24.986,11:00,4,"1 day, 1:00:00",myhome,c1,c2
1,2,121.443,24.433,12:00,5,"1 day, 1:00:00",apple,commute,c2
2,3,121.123,24.765,13:00\n,6,"1 day, 5:00:00",banana,commute,c2
3,4,121.522,25.063,18:00,3,1:14:00,LOUISA COFFEE 路易莎咖啡 (民權西門市),commute,restaurant
4,57,25.087,25.087,19:14,3,17:04:00,家,home,entering myPlace
5,164,121.563,25.087,12:18,4,0:34:00,家,commute,exiting myPlace
6,244,25.057,25.057,12:52,4,4:02:00,公司,work place,entering myPlace
7,267,121.616,25.057,16:54,4,0:28:00,公司,commute,exiting myPlace
8,374,25.087,25.087,17:22,4,1:30:00,家,home,entering myPlace
9,388,121.563,25.087,18:52,4,0:08:00,家,commute,exiting myPlace


## Map Category

In [96]:
stopLoc = pd.merge(stopLoc, loc_to_cat, how='left', on='category1' )
stopLoc = stopLoc.drop(['name1','category1','location_id'],1)
stopLoc['user_id'] = 2
stopLoc = stopLoc[['user_id','weekday','start_time','duration','location']]
stopLoc
#把start_date加回來
#name1,category1拿掉 place_cat對表換成數字

,user_id,weekday,start_time,duration,location
0,2,4,11:00,"1 day, 1:00:00",NaN
1,2,5,12:00,"1 day, 1:00:00",NaN
2,2,6,13:00\n,23:10:00,NaN
3,2,6,12:10,5:58:00,5.0
4,2,6,18:08,0:01:00,NaN
5,2,6,18:09,0:20:00,1.0
6,2,6,18:29,0:14:00,NaN
7,2,6,18:43,0:34:00,5.0
8,2,6,19:17,0:58:00,19.0
9,2,6,20:15,0:23:00,NaN


In [ ]:
traing = stopLoc.iloc[:, np.r_[0:4]].values
predictions = clf.predict(traing)

## Myplace

In [ ]:
savedplace = pd.read_csv("savedPlace.csv")
savedplace['place_longitude'] = round(saveplace['place_longitude'],3)
savedplace['place_latitude'] = round(saveplace['place_latitude'],3)
#savedplace = savedplace.loc[savedplace['my_place'] == 1]
savedplace = savedplace.drop(['place_name','my_place','place_id'],1)
savedplace = savedplace.rename(columns={"place_longitude": "longitude", "place_latitude": "latitude"})
savedplace = savedplace.drop_duplicates(subset=['longitude', 'latitude'], keep='last')
savedplace.head()

In [ ]:
stopLoc = pd.merge(stopLoc, savedplace, how='left', on=['longitude','latitude'])
stopLoc